In [ ]:
import json
import urllib.request
import collections

In [ ]:
def find_all_data(users, token):
    users_data = dict()
    for user in users:
        counter = 1
        in_dict = users_data.setdefault(user, [])
        while True:
            url = 'https://api.github.com/users/%s/repos?access_token=%s&page=%d' % (user, token, counter)
            response = urllib.request.urlopen(url)
            text = response.read().decode('utf-8')
            data = json.loads(text)
            if data == []:
                break
            in_dict += data
            counter += 1
    return users_data

def get_username(users):
    print('Выберите одного пользователя из списка.')
    username = False
    
    while username == False:
        username = input('Введите название репозитория: ')
        if username not in users:
            username = False
            print('Такого пользователя нет в списке, введите другое имя: ')
            
    return username

def get_languages(username, users_data):
    lang_in_rep = dict()
    for rep in users_data[username]:
        if rep['language'] == None:
            continue
        in_dict = lang_in_rep.setdefault(rep['language'], [])
        in_dict.append(rep['name'])
    return lang_in_rep
        
def print_user_langs(langs):
    j_langs = ', '.join(langs.keys())
    print('В репозитории используются языки: %s.\n' % j_langs)
    for lang, rep in langs.items():
        print('Язык %s используется в %s, всего репозиториев: %d.' % (lang, ', '.join(rep), len(rep)))

def has_more_reps(usernames, users_data):
    rep_number = 0
    the_name = ''
    for name in usernames:
        number = len(users_data[name])
        if number > rep_number:
            rep_number = number
            the_name = name
    return the_name, rep_number

def popular_lang(usernames, users_data):
    all_langs_numb = collections.Counter()
    for name in usernames:
        all_langs = get_languages(name, users_data)
        all_langs_numb_user = collections.Counter()
        for key in all_langs.keys():
            all_langs_numb_user[key] = len(all_langs[key])
        all_langs_numb += all_langs_numb_user
    return all_langs_numb.most_common(1)[0]

def find_all_folls(users, token):
    number = 0
    name = ''
    for user in users:
        counter = 1
        foll_list = []
        while True:
            url = 'https://api.github.com/users/%s/followers?access_token=%s&page=%d' % (user, token, counter)
            response = urllib.request.urlopen(url)
            text = response.read().decode('utf-8')
            data = json.loads(text)
            if data == []:
                lenth = len(foll_list)
                break
            foll_list += data
            counter += 1
        if lenth > number:
            number = lenth
            name = user        
    return name, number


def main():
    users = ('elmiram','maryszmary','lizaku',
    'nevmenandr','ancatmara','roctbb','akutuzov',
    'agricolamz','lehkost','kylepjohnson','mikekestemont',
    'demidovakatya','shwars','JelteF','timgraham',
    'arogozhnikov','jasny','bcongdon','whyisjake')
    
    token = 'bedad35272c72fd889b2b3e351b64b2a003b19ac' #Вставьте в переменную ваш токен, если GitHub вас банит!
    
    users_data = find_all_data(users, token)
    
    print('Перед вами список имён пользователей GitHub\'а:')
    print(*users, sep=', ')
    print('\n')
    
    name = get_username(users)
    print_user_langs(get_languages(name, users_data))
    print('\n')
    
    the_name, number = has_more_reps(users, users_data)
    print('Наибольшее количество репозиториев (%s) у пользователя %s.\n' % (number, the_name))
    
    lang, lang_freq = popular_lang(users, users_data)
    print('Наибольшее количество раз (%d) во всех репозиториях встречается язык %s.\n' % (lang_freq, lang))
    
    followee_names, folls_num = find_all_folls(users, token)
    print('Наибольшее количество последователей (%d) у пользователя %s.' % (folls_num, followee_names))

In [ ]:
try:
    main()
except urllib.error.HTTPError:
    print('Введите ваш токен в переменную, отмеченную комментарием в тексте программы, и запустите её снова.')